# Quantum Algorithm Section

In [2]:
import pennylane as qml
import numpy as np

## A.1
Fill in the following code to create the uniform superposition over n qubits.
It will plot the probability of observing different outcomes. Does putting the circuit in a uniform superposition help us break the lock

In [3]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def naive_circuit():
    """Create a uniform superposition and return the probabilities.

    Returns: 
        array[float]: Probabilities for observing different outcomes.
    """
    for wire in range(n_bits):

        ##################
        qml.Hadamard(wires = wire)

    return qml.probs(wires=range(n_bits))


## Codercise A.2.1. 
Write a function which returns the oracle in matrix form for a given secret combination.



In [7]:
def oracle_matrix(combo):
    """Return the oracle matrix for a secret combination.
    
    Args:
        combo (list[int]): A list of bits representing a secret combination.
         
    Returns: 
        array[float]: The matrix representation of the oracle.
    """
    index = np.ravel_multi_index(combo, [2]*len(combo)) # Index of solution ,[0],[2]
    print("Index", index)
    my_array = np.identity(2**len(combo)) # Create the identity matrix

    ##################
    # YOUR CODE HERE #
    ##################

    # MODIFY DIAGONAL ENTRY CORRESPONDING TO SOLUTION INDEX

    return my_array


In [11]:
combo = [1,0]
oracle_matrix(combo=combo)

Index 2


array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [24]:
a = np.arange(1,17).reshape(4,4)
print(a)
print("-"*10)
np.ravel_multi_index([2,7],[10,10])

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
----------


27

In [29]:
[2] * 4

[2, 2, 2, 2]